In [ ]:



#| notest
#| eval: false
#| hide

import sys

sys.path.insert(0, '../llm_sandbox_ui/')
sys.path.insert(0, '../../../')

import big_project_helper as bph

bph.display_project_contents('llm_sandbox_ui')

Dialog Name: unreal/unreal-llm-sandbox/nbs/streaming

../llm_sandbox_ui/cells.py
# AUTOGENERATED! DO NOT EDIT! File to edit: ../nbs/cells.ipynb.

# %% auto 0
__all__ = ['up_arrow_ic', 'down_arrow_ic', 'close_ic', 'swap_ic', 'view_ic', 'clean_ic', 'minimize_ic', 'play_ic', 'stop_ic',
           'edit_ic', 'tools_ic', 'label_css', 'cell_button_format', 'interrupt_button', 'BaseCell', 'PromptCell',
           'MarkdownCell', 'CodeCell']

# %% ../nbs/cells.ipynb 3
import json
import uuid
import re
import mistune
from fasthtml.common import * 
from .app_config import PROMPT_SPLIT#, AGENT_CODE_SPLIT


# %% ../nbs/cells.ipynb 4
up_arrow_ic = NotStr("&#11014")
down_arrow_ic = NotStr("&#11015")
close_ic = NotStr("&#x274C")
swap_ic = NotStr("&#128257")
view_ic =NotStr("&#x1F50D")
clean_ic =NotStr("&#x1F9F9")
minimize_ic = NotStr("&#x25BC");
play_ic = NotStr("&#9654")
stop_ic = NotStr("&#9209")
edit_ic = NotStr("&#x1F4DD")
tools_ic = NotStr("&#x1F6E0")

label_css = "text-xs text-gray-400 px-2 py-

In [ ]:
#| default_exp streaming


# Streaming

In [ ]:
#| export
import queue
import asyncio
import json
from starlette.responses import StreamingResponse


active_streams = {}


class SSEStream:
    """Unified SSE streaming with queue-based message passing.
    
    Args:
        cell_id: Unique identifier for abort handling.
    """
    def __init__(self, stream_key):
        self.stream_key = stream_key  
        self.q = queue.Queue()
        active_streams[stream_key] = {'abort': False}
    
    def text(self, content: str):
        """Send text chunk to stream.
        
        Args:
            content: String to send.
        """
        self.q.put({"type": "text", "data": content})

    def tag(self, content: str):
        """Send control tag to switch stream target.
        
        Args:
            content: Tag name.
        """
        self.q.put({"type": "tag", "data": content})
    
    def output(self, data):
        """Send Jupyter-style output dict.
        
        Args:
            data: Output dict/list (may contain ANSI).
        """
        self.q.put({"type": "output", "data": data})
        
    def done(self):
        """Signal stream complete and cleanup."""
        self.q.put(None)
        self.cleanup()

    def aborted(self):
        """Check if user requested abort.
        
        Returns:
            True if aborted, False otherwise.
        """
        return active_streams.get(self.stream_key, {}).get('abort', True)

    def response(self):
        """Create async SSE response.
        
        Returns:
            StreamingResponse for FastHTML route.
        """
        async def generator():
            while True:
                item = await asyncio.to_thread(self.q.get)
                if item is None: break
                yield f"data: {json.dumps(item)}\n\n"
            self.cleanup()
        return StreamingResponse(generator(), media_type='text/event-stream')

    def cleanup(self):
        """Remove cell_id from active_streams."""
        active_streams.pop(self.stream_key, None)
        